#### HOTELS ATTRIBUTES EXTRACTION PROCESS

This notebook was used to execute manually the scripts to scrape all of the previously scrapped hotels attributes.

From the scrapping.py file we import scrape_hotels_attributes, this function takes a list of urls and returns a dataframe with the following hotel data:

- attributes
- scores

The data was scrapped from the [Booking.com](https://www.booking.com/index.es.html?label=gen173nr-1BCAEoggI46AdIM1gEaAyIAQGYAQq4ARfIAQzYAQHoAQGIAgGoAgO4At_puqsGwAIB0gIkOTkxOTFjMmEtNTEzYS00NDQyLTgxNmMtMWJmMzMyYzVjNjJj2AIF4AIB&sid=1a3dea14733e69dc474ff74d1fad21e9&keep_landing=1&sb_price_type=total&) web, from a page like this: [boooking/hotels/newyork/hotel.com](https://www.booking.com/hotel/us/west-side-ymca.es.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaAyIAQGYAQq4ARfIAQzYAQHoAQH4AQyIAgGoAgO4Arj9uqsGwAIB0gIkMzNmMDQ2ZDMtOTkwYy00ZWVmLTgwZDYtYzhjMWUxZmFjM2Yz2AIG4AIB&sid=75f60e0ba4b489f2fe2baf48281729c2&dest_id=20088325;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;hpos=4;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1701756621;srpvid=525f2b61586300dd;type=total;ucfs=1&#hotelTmpl).

In [7]:
import sys
import scrapping
import pandas as pd

usa_cities = pd.read_csv("../files/data/usa_cities.csv")
usa_states = pd.read_csv("../files/data/usa_states.csv")

# This functions takes a state_id and executes the scrapping process
def scrape(state_id: list):
    state = state=usa_states["state"][usa_states["state_id"] == state_id].iloc[0]
    hotels = pd.read_csv(f"../files/data/booking/{state}_hotels.csv")
    hotels_attributes = scrapping.scrape_hotels_attributes(hotels["reviews_url"].tolist())
    # personalised_graphics.missing_values_heatmap(hotels_attributes, f"{state}_hotels_attributes scrape result") # explore data
    return hotels_attributes

Utah

In [14]:
utah_hotels = pd.read_csv("../files/data/booking/utah_hotels.csv")
hotels_attributes = scrapping.scrape_hotels_attributes(utah_hotels["reviews_url"].head(10).tolist())
hotels_attributes.to_csv("../files/data/booking/california_hotels_attributes_sample10.csv")

1/10
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\anasi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\anasi\AppData\Local\Temp\ipykernel_20808\957686571.py", line 2, in <module>
    hotels_attributes = scrapping.scrape_hotels_attributes(utah_hotels["reviews_url"].head(10).tolist())
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anasi\OneDrive\Documentos\GitHub\PF_DS_YELP\scrapping\scrapping.py", line 188, in scrape_hotels_attributes
  File "C:\Users\anasi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\selenium\webdriver\remote\webdriver.py", line 353, in get
    self.execute(Command.GET, {"url": url})
  File "C:\Users\anasi\AppData